In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-08-08 01:12:53 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-08-08 01:12:53 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-08-08 01:12:53 __main__ INFO     transformers.__version__='4.54.1'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-08-08 01:12:56 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-08-08 01:12:56 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-08-08 01:12:57 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-08-08 01:12:57 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-08-08 01:12:58 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-08-08 01:12:58 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-08-08 01:12:58 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-08-08 01:12:58 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-08-08 01:12:58 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/meta-llama/Llama-3.3-70B-Instruct/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-08-08 01:13:47 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-08-08 01:13:47 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0
2025-08-08 01:13:47 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [5]:
from src.selection.data import SelectOneTask

select_prof = SelectOneTask.load(
    path=os.path.join(env_utils.DEFAULT_DATA_DIR, "selection", "profession.json")
)

print(select_prof)

SelectOneTask: (profession of a famous person)
Categories: actor(20), singer(20), comedian(20), director(20), basketball player(20), football player(20), soccer player(20), tennis player(20), golfer(20), boxer(20), news anchor(20), journalist(20), author(20), fashion designer(20), entrepreneur(19), politician(20)



In [6]:
sample = select_prof.get_random_sample(
    mt = mt,
    prompt_template_idx=2,
    option_style="numbered",
    category="actor"
)

print(sample)

Leonardo DiCaprio -> Chris Hemsworth (4): ['Kawhi Leonard', 'Andrey Rublev', 'Barack Obama', 'Errol Spence Jr.', 'Chris Hemsworth', 'Vinícius Júnior']


In [7]:
# sample.prompt_template = select_prof.prompt_templates[3]

print(f'"{sample.prompt()}"', ">>", sample.obj)

"Which person from the following list is by profession a actor?
1. Kawhi Leonard
2. Andrey Rublev
3. Barack Obama
4. Errol Spence Jr.
5. Chris Hemsworth
6. Vinícius Júnior
Answer:" >> Chris Hemsworth


In [8]:
print(sample.prompt(option_style="single_line"))

Which person from the following list is by profession a actor?
Options: Kawhi Leonard, Andrey Rublev, Barack Obama, Errol Spence Jr., Chris Hemsworth, Vinícius Júnior.
Answer:


In [9]:
from src.functional import generate_with_patch

gen = generate_with_patch(
    mt = mt,
    inputs = sample.prompt(),
    max_new_tokens=20,
    do_sample=False,
    remove_prefix=True
)[0]
print(f'"{gen}"', ">>", sample.obj)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


" Chris Hemsworth
Explanation: Chris Hemsworth is an Australian actor. He rose to prominence playing Kim" >> Chris Hemsworth


In [10]:
from src.attention import get_attention_matrices
from src.experiments.utils import (
    get_patches_to_verify_independent_enrichment,
    verify_head_patterns,
)

HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

attn_pattern = verify_head_patterns(
    prompt = sample.prompt(),
    options = sample.options,
    pivot = sample.subj,
    mt = mt,
    heads = HEADS
)

0 patches to ablate possible answer information from options


dict_keys(['loss', 'logits', 'past_key_values', 'hidden_states', 'attentions'])
2025-08-08 01:13:58 src.experiments.utils DEBUG    Predictions: ['" Chris"[11517] (p=0.578, logit=20.875)', '" "[220] (p=0.188, logit=19.750)', '" The"[578] (p=0.146, logit=19.500)', '" Option"[7104] (p=0.025, logit=17.750)', '" ("[320] (p=0.017, logit=17.375)']
2025-08-08 01:13:58 src.experiments.utils INFO     Combined attention matrix for all heads


In [11]:
select_prof.category_wise_examples.keys()

dict_keys(['actor', 'singer', 'comedian', 'director', 'basketball player', 'football player', 'soccer player', 'tennis player', 'golfer', 'boxer', 'news anchor', 'journalist', 'author', 'fashion designer', 'entrepreneur', 'politician'])

In [45]:
import copy
import random
from src.selection.utils import KeyedSet, get_first_token_id
from src.selection.data import SelectionSample
from src.functional import predict_next_token

######################################################################
N_DISTRACTORS = 5
WINDOW_SPEC = {
    mt.layer_name_format: 1,
    mt.mlp_module_name_format: 9,
    mt.attn_module_name_format: 9,
}
module_name_format = mt.layer_name_format
# module_name_format = mt.mlp_module_name_format
# module_name_format = mt.attn_module_name_format

######################################################################


def get_counterfactual_samples_on_pivot_entity(
    task: SelectOneTask = select_prof,
    patch_category: str | None = None,
    clean_category: str | None = None,
    shuffle_clean_options: bool = False,
    prompt_template_idx=2,
    option_style="numbered",
    filter_by_lm_prediction: bool = True,
):
    categories = list(task.category_wise_examples.keys())
    if patch_category is None:
        patch_category = random.choice(categories)

    patch_subj, patch_obj = random.sample(
        task.category_wise_examples[patch_category], 2
    )
    logger.info(
        f"Patch category: {patch_category}, subject: {patch_subj}, object: {patch_obj}"
    )

    if clean_category is None:
        clean_category = random.choice(list(set(categories) - {patch_category}))

    clean_options = task.category_wise_examples[clean_category]
    random.shuffle(clean_options)

    clean_subj, clean_obj = random.sample(
        (KeyedSet(clean_options, mt.tokenizer) - KeyedSet([patch_obj], mt.tokenizer)).values, 2
    )
    logger.info(
        f"Clean category: {clean_category}, subject: {clean_subj}, object: {clean_obj}"
    )

    distractors = []
    other_categories = random.sample(
        list(set(categories) - {patch_category, clean_category}),
        k=N_DISTRACTORS - 1,
    )

    for other_category in other_categories:
        other_examples = task.category_wise_examples[other_category]
        random.shuffle(other_examples)
        other_examples = KeyedSet(other_examples, mt.tokenizer)
        distractors.append(
            random.choice(
                (
                    other_examples
                    - KeyedSet(
                        [patch_obj, clean_obj] + distractors, tokenizer=mt.tokenizer
                    )
                ).values
            )
        )

    patch_options = [patch_obj, clean_obj] + distractors
    random.shuffle(patch_options)
    patch_obj_idx = patch_options.index(patch_obj)
    logger.info(f"{patch_obj_idx=} | {patch_options}")

    clean_options = copy.deepcopy(patch_options)

    if shuffle_clean_options:
        # Useful for the pointer experiments
        while (
            clean_options.index(clean_obj) == patch_obj_idx
            or clean_options.index(patch_obj) == patch_obj_idx
        ):
            random.shuffle(clean_options)

    clean_obj_idx = clean_options.index(clean_obj)

    logger.info(f"{clean_obj_idx=} | {clean_options}")

    kwargs = dict(
        prompt_template= task.prompt_templates[prompt_template_idx],
        default_option_style=option_style,
    )

    patch_sample = SelectionSample(
        subj=patch_subj,
        obj=patch_obj,
        obj_idx=patch_obj_idx,
        obj_token_id=get_first_token_id(patch_obj, mt.tokenizer, prefix=" "),
        options=patch_options,
        category=patch_category,
        **kwargs,
    )
    clean_sample = SelectionSample(
        subj=clean_subj,
        obj=clean_obj,
        obj_idx=clean_obj_idx,
        obj_token_id=get_first_token_id(clean_obj, mt.tokenizer, prefix=" "),
        options=clean_options,
        category=clean_category,
        **kwargs,
    )

    if filter_by_lm_prediction:
        for sample in [patch_sample, clean_sample]:
            pred = predict_next_token(
                mt=mt,
                inputs=sample.prompt(),
            )[0]
            logger.info(f"{sample.subj} -> {sample.obj} | pred={[str(p) for p in pred]}")
            if pred[0].token_id != sample.obj_token_id:
                logger.error(
                    f'Prediction mismatch: {pred[0].token_id}["{mt.tokenizer.decode(pred[0].token_id)}"] != {sample.obj_token_id}["{mt.tokenizer.decode(sample.obj_token_id)}"]'
                )
                return get_counterfactual_samples_on_pivot_entity(
                    task=task,
                    patch_category=patch_category,
                    clean_category=clean_category,
                    shuffle_clean_options=shuffle_clean_options,
                    prompt_template_idx=prompt_template_idx,
                    option_style=option_style,
                    filter_by_lm_prediction=filter_by_lm_prediction,
                )
            sample.prediction = pred

    return patch_sample, clean_sample

In [46]:
patch_sample, clean_sample = get_counterfactual_samples_on_pivot_entity(
    patch_category="politician",
    clean_category="actor",
    filter_by_lm_prediction=False,
    prompt_template_idx=3
)

2025-08-08 01:17:29 __main__ INFO     Patch category: politician, subject: Marco Rubio, object: Bernie Sanders
2025-08-08 01:17:29 __main__ INFO     Clean category: actor, subject: Meryl Streep, object: Julia Roberts
2025-08-08 01:17:30 __main__ INFO     patch_obj_idx=1 | ['Julia Roberts', 'Bernie Sanders', 'Dmitry Bivol', 'Simona Halep', 'Gillian Flynn', 'Tim Cook']
2025-08-08 01:17:30 __main__ INFO     clean_obj_idx=0 | ['Julia Roberts', 'Bernie Sanders', 'Dmitry Bivol', 'Simona Halep', 'Gillian Flynn', 'Tim Cook']


In [47]:
from src.functional import generate_with_patch

for sample in [patch_sample, clean_sample]:
    print(sample.prompt(), ">>", sample.obj)
    attn_pattern = verify_head_patterns(
        prompt=sample.prompt(),
        options=sample.options,
        pivot=sample.subj,
        mt=mt,
        heads=HEADS,
        generate_full_answer=True
    )

1. Julia Roberts
2. Bernie Sanders
3. Dmitry Bivol
4. Simona Halep
5. Gillian Flynn
6. Tim Cook
Who among these people mentioned above is a politician by profession?
Answer: >> Bernie Sanders
0 patches to ablate possible answer information from options
2025-08-08 01:17:34 src.experiments.utils DEBUG    Generated full answer: " Bernie Sanders
The best answer is Bernie Sanders."
dict_keys(['loss', 'logits', 'past_key_values', 'hidden_states', 'attentions'])
2025-08-08 01:17:34 src.experiments.utils DEBUG    Predictions: ['" Bernie"[30324] (p=0.684, logit=21.125)', '" Among"[22395] (p=0.056, logit=18.625)', '" ("[320] (p=0.050, logit=18.500)', '" Option"[7104] (p=0.050, logit=18.500)', '" The"[578] (p=0.039, logit=18.250)']
2025-08-08 01:17:34 src.experiments.utils INFO     Combined attention matrix for all heads


1. Julia Roberts
2. Bernie Sanders
3. Dmitry Bivol
4. Simona Halep
5. Gillian Flynn
6. Tim Cook
Who among these people mentioned above is a actor by profession?
Answer: >> Julia Roberts
0 patches to ablate possible answer information from options
2025-08-08 01:17:38 src.experiments.utils DEBUG    Generated full answer: " Julia Roberts
Explanation: Julia Roberts is an American actress and producer. She is known for her roles in movies like "Pretty Woman", "Erin"
dict_keys(['loss', 'logits', 'past_key_values', 'hidden_states', 'attentions'])
2025-08-08 01:17:38 src.experiments.utils DEBUG    Predictions: ['" Julia"[40394] (p=0.895, logit=22.375)', '" Among"[22395] (p=0.021, logit=18.625)', '" The"[578] (p=0.019, logit=18.500)', '" Only"[8442] (p=0.016, logit=18.375)', '" Option"[7104] (p=0.014, logit=18.250)']
2025-08-08 01:17:38 src.experiments.utils INFO     Combined attention matrix for all heads


In [48]:
import baukit
from src.functional import get_module_nnsight, PatchSpec
from src.hooking.llama_attention import LlamaAttentionPatcher
import types
from typing import Literal
from src.tokens import prepare_input


def set_attn_implementation(mt, attn_implementation: Literal["sdpa", "eager"]):
    mt.config._attn_implementation = attn_implementation
    for layer_idx in range(mt.config.num_hidden_layers):
        attn_block_name = mt.attn_module_name_format.format(layer_idx)
        attn_block = baukit.get_module(mt._model, attn_block_name)
        attn_block.config._attn_implementation = attn_implementation


###################################################################################
batch_size = 1  # tokenized.input_ids.shape[0]
n_heads = mt.config.num_attention_heads
head_dim = mt.n_embd // n_heads
query_idx = -1 # almost always the last token
###################################################################################

mt.reset_forward()
set_attn_implementation(mt, "sdpa")

layer_idx, head_idx = HEADS[0]

attn_block_name = mt.attn_module_name_format.format(layer_idx)
attn_block = baukit.get_module(mt._model, attn_block_name)
attn_block.forward = types.MethodType(
    LlamaAttentionPatcher(block_name=attn_block_name),
    attn_block,
)

patch_tokenized = prepare_input(prompts=patch_sample.prompt(), tokenizer=mt)
patch_seq_len = patch_tokenized.input_ids.shape[1]
input_ln = mt.layer_name_format.format(layer_idx) + ".input_layernorm"

with mt.trace(patch_tokenized) as trace:
    ln_module = get_module_nnsight(mt, input_ln)
    patch_ln = ln_module.output.save()

    q_proj_name = mt.attn_module_name_format.format(layer_idx) + ".q_proj"
    q_proj_module = get_module_nnsight(mt, q_proj_name)
    patch_q_proj = q_proj_module.output.view(batch_size, patch_seq_len, n_heads, head_dim).transpose(1, 2).save()
    # patch_q_proj = PatchSpec(
    #     location=(q_proj_name + f".{head_idx}", -1),
    #     patch=patch_q_proj[:, head_idx, query_idx, :].squeeze().save()
    # )

clean_tokenized = prepare_input(prompts=clean_sample.prompt(), tokenizer=mt)
clean_seq_len = clean_tokenized.input_ids.shape[1]
with mt.trace(clean_tokenized) as trace:
    ln_module = get_module_nnsight(mt, input_ln)
    clean_ln = ln_module.output.save()

    q_proj_name = mt.attn_module_name_format.format(layer_idx) + ".q_proj"
    q_proj_module = get_module_nnsight(mt, q_proj_name)
    clean_q_proj = q_proj_module.output.view(batch_size, clean_seq_len, n_heads, head_dim).transpose(1, 2).save()
    # clean_q_proj = PatchSpec(
    #     location=(q_proj_name + f".{head_idx}", -1),
    #     patch=clean_q_proj[:, head_idx, query_idx, :].squeeze().save()
    # )

mt.reset_forward()
set_attn_implementation(mt, "eager")

patch_q_proj.shape, clean_q_proj.shape, patch_ln.shape, clean_ln.shape

2025-08-08 01:17:44 src.hooking.llama_attention DEBUG    LlamaAttentionPatcher <> model.layers.33.self_attn
2025-08-08 01:17:44 src.hooking.llama_attention DEBUG    hidden_shape=(1, 49, -1, 128) | input_shape=torch.Size([1, 49]) | torch.Size([1, 49, 8192])
2025-08-08 01:17:44 src.hooking.llama_attention DEBUG    query_states.size()=torch.Size([1, 64, 49, 128]) | key_states.size()=torch.Size([1, 8, 49, 128]) | value_states.size()=torch.Size([1, 8, 49, 128])
2025-08-08 01:17:44 src.hooking.llama_attention DEBUG    LlamaAttentionPatcher <> model.layers.33.self_attn
2025-08-08 01:17:44 src.hooking.llama_attention DEBUG    hidden_shape=(1, 49, -1, 128) | input_shape=torch.Size([1, 49]) | torch.Size([1, 49, 8192])
2025-08-08 01:17:44 src.hooking.llama_attention DEBUG    query_states.size()=torch.Size([1, 64, 49, 128]) | key_states.size()=torch.Size([1, 8, 49, 128]) | value_states.size()=torch.Size([1, 8, 49, 128])


(torch.Size([1, 64, 49, 128]),
 torch.Size([1, 64, 49, 128]),
 torch.Size([1, 49, 8192]),
 torch.Size([1, 49, 8192]))

In [49]:
from src.attention import visualize_attn_matrix
from src.functional import get_hs, interpret_logits

mt.reset_forward()
set_attn_implementation(mt, "sdpa")

layer_idx, head_idx = HEADS[0]

attn_matrices = {layer_idx: {}}

attn_block_name = mt.attn_module_name_format.format(layer_idx)
attn_block = baukit.get_module(mt._model, attn_block_name)
attn_block.forward = types.MethodType(
    LlamaAttentionPatcher(
        block_name=attn_block_name,
        save_attn_for=[head_idx],
        store_attn_matrices=attn_matrices[layer_idx],
    ),
    attn_block,
)

logit_location = (mt.lm_head_name, -1)
logits = get_hs(
    mt = mt,
    input = clean_tokenized,
    locations = [logit_location],
    return_dict=False
).squeeze()  # (seq_len, vocab_size)

mt.reset_forward()
set_attn_implementation(mt, "eager")

head_matrix = attn_matrices[layer_idx][head_idx].squeeze().to(torch.float32).cpu().numpy()

visualize_attn_matrix(
    attn_matrix=head_matrix,
    tokens=[mt.tokenizer.decode(t) for t in clean_tokenized.input_ids[0]],
    q_index=-1,
)

interpret_logits(
    tokenizer=mt,
    logits=logits,
    interested_tokens=[patch_sample.obj_token_id, clean_sample.obj_token_id]
)

2025-08-08 01:17:45 src.hooking.llama_attention DEBUG    LlamaAttentionPatcher <> model.layers.33.self_attn
2025-08-08 01:17:45 src.hooking.llama_attention DEBUG    hidden_shape=(1, 49, -1, 128) | input_shape=torch.Size([1, 49]) | torch.Size([1, 49, 8192])
2025-08-08 01:17:45 src.hooking.llama_attention DEBUG    query_states.size()=torch.Size([1, 64, 49, 128]) | key_states.size()=torch.Size([1, 8, 49, 128]) | value_states.size()=torch.Size([1, 8, 49, 128])


([PredictedToken(token=' Julia', prob=0.88671875, logit=22.375, token_id=40394, metadata=None),
  PredictedToken(token=' Among', prob=0.023681640625, logit=18.75, token_id=22395, metadata=None),
  PredictedToken(token=' The', prob=0.0208740234375, logit=18.625, token_id=578, metadata=None),
  PredictedToken(token=' Only', prob=0.0162353515625, logit=18.375, token_id=8442, metadata=None),
  PredictedToken(token=' Option', prob=0.01434326171875, logit=18.25, token_id=7104, metadata=None)],
 {40394: (1,
   PredictedToken(token=' Julia', prob=0.88671875, logit=22.375, token_id=40394, metadata=None)),
  30324: (249,
   PredictedToken(token=' Bernie', prob=1.8849968910217285e-06, logit=9.3125, token_id=30324, metadata=None))})

In [50]:
from src.attention import visualize_attn_matrix

mt.reset_forward()
set_attn_implementation(mt, "sdpa")

layer_idx, head_idx = HEADS[0]

attn_matrices = {layer_idx: {}}

attn_block_name = mt.attn_module_name_format.format(layer_idx)
attn_block = baukit.get_module(mt._model, attn_block_name)
attn_block.forward = types.MethodType(
    LlamaAttentionPatcher(
        block_name=attn_block_name,
        save_attn_for=[head_idx],
        store_attn_matrices=attn_matrices[layer_idx],
        query_patches=[[head_idx, query_idx, patch_q_proj[:, head_idx, query_idx, :].squeeze()]],
    ),
    attn_block,
)

logit_location = (mt.lm_head_name, -1)
patch_logits = get_hs(
    mt = mt,
    input = clean_tokenized,
    locations = [logit_location],
    return_dict=False
).squeeze()  # (seq_len, vocab_size)

mt.reset_forward()
set_attn_implementation(mt, "eager")

head_matrix = attn_matrices[layer_idx][head_idx].squeeze().to(torch.float32).cpu().numpy()

visualize_attn_matrix(
    attn_matrix=head_matrix,
    tokens=[mt.tokenizer.decode(t) for t in clean_tokenized.input_ids[0]],
    q_index=-1,
)

interpret_logits(
    tokenizer=mt,
    logits=patch_logits,
    interested_tokens=[patch_sample.obj_token_id, clean_sample.obj_token_id]
)

2025-08-08 01:17:46 src.hooking.llama_attention DEBUG    LlamaAttentionPatcher <> model.layers.33.self_attn
2025-08-08 01:17:46 src.hooking.llama_attention DEBUG    hidden_shape=(1, 49, -1, 128) | input_shape=torch.Size([1, 49]) | torch.Size([1, 49, 8192])
2025-08-08 01:17:46 src.hooking.llama_attention DEBUG    query_states.size()=torch.Size([1, 64, 49, 128]) | key_states.size()=torch.Size([1, 8, 49, 128]) | value_states.size()=torch.Size([1, 8, 49, 128])


([PredictedToken(token=' Julia', prob=0.89453125, logit=22.375, token_id=40394, metadata=None),
  PredictedToken(token=' Among', prob=0.0238037109375, logit=18.75, token_id=22395, metadata=None),
  PredictedToken(token=' The', prob=0.0185546875, logit=18.5, token_id=578, metadata=None),
  PredictedToken(token=' Only', prob=0.016357421875, logit=18.375, token_id=8442, metadata=None),
  PredictedToken(token=' Option', prob=0.01446533203125, logit=18.25, token_id=7104, metadata=None)],
 {40394: (1,
   PredictedToken(token=' Julia', prob=0.89453125, logit=22.375, token_id=40394, metadata=None)),
  30324: (190,
   PredictedToken(token=' Bernie', prob=3.129243850708008e-06, logit=9.8125, token_id=30324, metadata=None))})

In [51]:
# torch.allclose(patch_logits, logits, atol = 1e-3)

In [52]:
# torch.allclose(
#     patch_q_proj[:, head_idx, query_idx, :], 
#     clean_q_proj[:, head_idx, query_idx, :],
#     atol=1e-3
# )

In [53]:
# torch.allclose(
#     patch_ln[:, query_idx, :], 
#     clean_ln[:, query_idx, :],
#     atol=1e-3
# )

In [54]:
# patch_ln[:, query_idx, :], clean_ln[:, query_idx, :]

In [55]:
# # manual calculation
# attn_module = baukit.get_module(mt._model, mt.attn_module_name_format.format(layer_idx))
# patch_q_proj_manual = attn_module.q_proj(patch_ln)
# clean_q_proj_manual = attn_module.q_proj(clean_ln)

# print(patch_q_proj_manual.shape, clean_q_proj_manual.shape)
# print(torch.allclose(
#     patch_q_proj_manual[:, query_idx, :], 
#     clean_q_proj_manual[:, query_idx, :],
#     atol=1e-3
# ))

# patch_q_proj_manual = patch_q_proj_manual.reshape(batch_size, patch_seq_len, n_heads, head_dim).transpose(1, 2)
# clean_q_proj_manual = clean_q_proj_manual.reshape(batch_size, clean_seq_len, n_heads, head_dim).transpose(1, 2)
# print(patch_q_proj_manual.shape, clean_q_proj_manual.shape)

# for idx in range(n_heads):
#     print(head_idx, torch.allclose(
#         patch_q_proj_manual[:, idx, query_idx, :], 
#         clean_q_proj_manual[:, idx, query_idx, :],
#         atol=1e-3
#     ))

In [56]:
# torch.allclose(
#     patch_q_proj_manual[:, head_idx, query_idx, :], 
#     patch_q_proj[:, head_idx, query_idx, :],
#     atol=1e-3
# )

In [57]:
replace_q_proj = clean_q_proj.clone()
replace_q_proj[:, head_idx, query_idx, :] = patch_q_proj[:, head_idx, query_idx, :]

print(torch.allclose(
        replace_q_proj[:, head_idx, query_idx, :],
        clean_q_proj[:, head_idx, query_idx, :],
        atol=1e-3
    )
)
print(replace_q_proj.shape)

replace_q_proj = replace_q_proj.transpose(1, 2).reshape(batch_size, clean_seq_len, -1)

rep_patch = PatchSpec(
    location=(q_proj_name, -1),
    patch=replace_q_proj[:, -1, :].squeeze(),
)

ln_patch = PatchSpec(
    location=(input_ln, -1),
    patch=patch_ln[:, query_idx, :].squeeze(),
)

head_q_patch = PatchSpec(
    location=(q_proj_name, head_idx, -1),
    patch=patch_q_proj[:, head_idx, query_idx, :].squeeze(),
)

head_q_patch.location, rep_patch.location

False
torch.Size([1, 64, 49, 128])


(('model.layers.33.self_attn.q_proj', 45, -1),
 ('model.layers.33.self_attn.q_proj', -1))

In [58]:
# clean_q_proj_rs = clean_q_proj.view(batch_size, clean_seq_len, -1)
# patch_q_proj_rs = patch_q_proj.view(batch_size, clean_seq_len, -1)
# clean_q_proj_rs[:, -1, :].shape, patch_q_proj_rs[:, -1, :].shape

In [59]:
# torch.allclose(
#     replace_q_proj[:, -1, :].squeeze(), 
#     # patch_q_proj_rs[:, -1, :].squeeze(),
#     clean_q_proj_rs[:, -1, :].squeeze(), 
#     atol=1e-3
# )

In [60]:
patch_q_proj.shape, clean_q_proj.shape
# torch.allclose(patch_q_proj.patch, clean_q_proj.patch)

(torch.Size([1, 64, 49, 128]), torch.Size([1, 64, 49, 128]))

In [65]:
from src.attention import get_attention_matrices, visualize_attn_matrix
from src.functional import interpret_logits

clean_tokenized = prepare_input(prompts=clean_sample.prompt(), tokenizer=mt)

attn_info = get_attention_matrices(
    input=clean_tokenized,
    mt=mt,
)

attn_matrix = attn_info.attention_matrices[layer_idx, head_idx].squeeze()
visualize_attn_matrix(
    attn_matrix=attn_matrix,
    tokens=[mt.tokenizer.decode(t) for t in clean_tokenized.input_ids[0]],
    q_index=-1,
)

interpret_logits(
    tokenizer=mt,
    logits=attn_info.logits,
    interested_tokens=[patch_sample.obj_token_id, clean_sample.obj_token_id]
)

([PredictedToken(token=' Julia', prob=0.89453125, logit=22.375, token_id=40394, metadata=None),
  PredictedToken(token=' Among', prob=0.02099609375, logit=18.625, token_id=22395, metadata=None),
  PredictedToken(token=' The', prob=0.0185546875, logit=18.5, token_id=578, metadata=None),
  PredictedToken(token=' Only', prob=0.016357421875, logit=18.375, token_id=8442, metadata=None),
  PredictedToken(token=' Option', prob=0.01446533203125, logit=18.25, token_id=7104, metadata=None)],
 {40394: (1,
   PredictedToken(token=' Julia', prob=0.89453125, logit=22.375, token_id=40394, metadata=None)),
  30324: (236,
   PredictedToken(token=' Bernie', prob=2.0265579223632812e-06, logit=9.375, token_id=30324, metadata=None))})

In [69]:
from src.attention import get_attention_matrices, visualize_attn_matrix
from src.functional import patch_with_nnsight, patch_with_baukit

clean_tokenized = prepare_input(prompts=clean_sample.prompt(), tokenizer=mt)

patched_attn_info = get_attention_matrices(
    input=clean_tokenized,
    mt=mt,
    # patches=[ln_patch],
    patches = [head_q_patch],
    # patches = [rep_patch]
    # patch_interface=patch_with_nnsight
    patch_interface=patch_with_baukit
)

patched_attn_matrix = patched_attn_info.attention_matrices[layer_idx, head_idx].squeeze()
visualize_attn_matrix(
    attn_matrix=patched_attn_matrix,
    tokens=[mt.tokenizer.decode(t) for t in clean_tokenized.input_ids[0]],
    q_index=-1,
)

interpret_logits(
    tokenizer=mt,
    logits=patched_attn_info.logits,
    interested_tokens=[patch_sample.obj_token_id, clean_sample.obj_token_id]
)

([PredictedToken(token=' Julia', prob=0.890625, logit=22.375, token_id=40394, metadata=None),
  PredictedToken(token=' Among', prob=0.0238037109375, logit=18.75, token_id=22395, metadata=None),
  PredictedToken(token=' The', prob=0.0185546875, logit=18.5, token_id=578, metadata=None),
  PredictedToken(token=' Only', prob=0.016357421875, logit=18.375, token_id=8442, metadata=None),
  PredictedToken(token=' Option', prob=0.014404296875, logit=18.25, token_id=7104, metadata=None)],
 {40394: (1,
   PredictedToken(token=' Julia', prob=0.890625, logit=22.375, token_id=40394, metadata=None)),
  30324: (191,
   PredictedToken(token=' Bernie', prob=3.11434268951416e-06, logit=9.8125, token_id=30324, metadata=None))})